In [10]:
from langchain_core.documents import Document

documents = [
    Document(

        page_content="Dogs are great pets, known for loyality",
        metadata={"sources":"mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their ownn space",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets, known for requiring simple care",
        metadata={"source":"first-pets-doc"}
    )

]

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022C7FF71880>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022C06309850>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [13]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embedding=embeddings)


In [16]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='cc20674e-af7f-45d7-bc13-ad61c9164dfc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their ownn space'),
  0.9657319784164429),
 (Document(id='292a6cc1-a8b4-4cb4-9757-59326eb8ca21', metadata={'sources': 'mammal-pets-doc'}, page_content='Dogs are great pets, known for loyality'),
  1.4119594097137451),
 (Document(id='f2ed85a4-8274-4c0d-ad20-875dfc4e84db', metadata={'source': 'first-pets-doc'}, page_content='Goldfish are popular pets, known for requiring simple care'),
  1.7077372074127197)]

In [17]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='cc20674e-af7f-45d7-bc13-ad61c9164dfc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their ownn space')],
 [Document(id='292a6cc1-a8b4-4cb4-9757-59326eb8ca21', metadata={'sources': 'mammal-pets-doc'}, page_content='Dogs are great pets, known for loyality')]]

In [20]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)

retriever.batch(["cat", "dog"])

[[Document(id='cc20674e-af7f-45d7-bc13-ad61c9164dfc', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their ownn space')],
 [Document(id='292a6cc1-a8b4-4cb4-9757-59326eb8ca21', metadata={'sources': 'mammal-pets-doc'}, page_content='Dogs are great pets, known for loyality')]]

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question based on the context only.
{question}
Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about cats")
print(response.content)

Based on the context, here's what I can tell you about cats:

Cats are independent pets that often enjoy their own space.
